## Import Statements

In [1]:
import numpy as np
import skimage
import matplotlib.pyplot as plt
from tempfile import TemporaryFile
from PIL import Image, ImageEnhance
from os import listdir
import imghdr
from skimage.transform import rotate, AffineTransform, warp
from skimage import filters
from torchvision import transforms
import cv2
import scipy
import random

In [2]:
rmdir '../Data_full/.ipynb_checkpoints'

rmdir: ../Data_full/.ipynb_checkpoints: No such file or directory


## Augmentation functions with image array inputs

In [ ]:
### Load Data and Handle Preprocessing
# This code takes in a dataset and saves each image with each of the augmentations seperately

# Input: Folder with all the .png images
# Output: A Map of file names mapped to imgs
def loadImages(path):
  imagesList = listdir(path)
  imgs = {}
  for image in imagesList:
    if imghdr.what(path + image) == 'png':
      if (image[6].isalpha()): # only add 5 of each image, only add alphabetical values
        img = Image.open(path + image)
        imgs[image[0:len(image) - 4]] = img
  return imgs

# Convert png img array to array np arrays
# Input: PNG image array
# Output: a list of numpy images
def ImagesToArray(imgs):
  imgs_array = {}
  for imgName in imgs.keys():
    img_array = np.array(imgs[imgName])
    imgs_array[imgName] = img_array
  return imgs_array

# Zero pad all images
# Input: list of numpy images
# Output: numpy array of N 600x600 images with 3 channels
def shape600(imgs):
  reshaped_array = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    x_pad_width = (600 - img.shape[0])//2
    y_pad_width = (600 - img.shape[1])//2
    new_img = (np.pad(img, ((x_pad_width, x_pad_width + (img.shape[0])%2), (y_pad_width, y_pad_width+(img.shape[1]%2)), (0,0)), constant_values=img[0][0][0])).astype('uint8')
    reshaped_array[imgName] = new_img
    img_save = Image.fromarray(new_img, 'RGB')
    img_save.save('./FinalImages/' + imgName + '.png')
  return reshaped_array

def blur(imgs, sd=1):
  filtered_img = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    new_img = skimage.filters.gaussian(img, sigma=sd)
    filtered_img[imgName + '_blurred'] = new_img
    img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    img_save.save('./FinalImages/' + imgName + '_blurred' + '.png')
  return filtered_img

def Scale(imgs):
  scaled_images = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    ratio = random.choice([0.1, 0.2, 0.3, 0.4])
    x = int(ratio * 600 / 2)
    scaled = img[x:600-x, x:600-x]
    new_img = cv2.resize(scaled, dsize=(600, 600), interpolation=cv2.INTER_CUBIC)
    scaled_images[imgName + '_scaled'] = new_img
    img_save = Image.fromarray(new_img, 'RGB')
    img_save.save('./FinalImages/' + imgName + '_scaled' + '.png')
  return scaled_images
    

def Rotate30(imgs): 
  rot30_imgs = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    rand_dir = random.choice([-1, 1])
    new_img = rotate(img, rand_dir * 30) 
    rot30_imgs[imgName + '_rotated'] = new_img
    img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    img_save.save('./FinalImages/' + imgName + '_rotated' + '.png')
  return rot30_imgs

def VerticalFlip(imgs): 
  flip_imgs = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    new_img = np.fliplr(img)
    flip_imgs[imgName + '_flipped'] = new_img
    img_save = Image.fromarray(new_img, 'RGB')
    img_save.save('./FinalImages/' + imgName + '_fipped' + '.png')
  return flip_imgs

def Translation(imgs): 
  trans_imgs = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    rand_x = random.randrange(-150, 150, 50)
    rand_y = random.randrange(-150, 150, 50)
    transform = AffineTransform(translation=(rand_x,rand_y))
    new_img = warp(img,transform, mode="constant")  
    trans_imgs[imgName + '_flipped'] = new_img
    img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    img_save.save('./FinalImages/' + imgName + '_translated' + '.png')
  return trans_imgs
####################################################

# load png_images from Data_full foler
path = "./handgesturedataset_part1/"

# images in an array named imgs
imgs_png = loadImages(path)

# Step 1 convert png_images to np arrays
imgs_array_before = ImagesToArray(imgs_png)

# # Step 2 Reshape the images
imgs_array_reshaped = shape600(imgs_array_before)

# # Step 3 Blur the images
blurred_imgs = blur(imgs_array_reshaped, 10)

# Step 4 Scale the images
scaled_imgs = Scale(imgs_array_reshaped)

# Step 5 Flip the images
flipped_imgs = VerticalFlip(imgs_array_reshaped)

# Step 6 Add translation to images
translated_imgs = Translation(imgs_array_reshaped)

# Step 7 Rotate 30 Degrees
rotated30_imgs = Rotate30(imgs_array_reshaped)

## Augmentation functions with only one image input

In [4]:
### Load Data and Handle Preprocessing

# Input: Folder with all the .png images
# Output: A Map of file names mapped to imgs
def loadImages(path):
  imagesList = listdir(path)
  imgs = {}
  for image in imagesList:
    if imghdr.what(path + image) == 'png':
      if (image[6].isalpha()): # only add 5 of each image, only add alphabetical values
        img = Image.open(path + image)
        imgs[image[0:len(image) - 4]] = img
  return imgs

# Convert png img array to array np arrays
# Input: PNG image array
# Output: a list of numpy images
def ImagesToArray(imgs):
  imgs_array = {}
  for imgName in imgs.keys():
    img_array = np.array(imgs[imgName])
    imgs_array[imgName] = img_array
  return imgs_array

# Zero pad all images
# Input: list of numpy images
# Output: numpy array of N 600x600 images with 3 channels
def shape600(imgs):
  reshaped_array = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    x_pad_width = (600 - img.shape[0])//2
    y_pad_width = (600 - img.shape[1])//2
    new_img = (np.pad(img, ((x_pad_width, x_pad_width + (img.shape[0])%2), (y_pad_width, y_pad_width+(img.shape[1]%2)), (0,0)), constant_values=img[0][0][0])).astype('uint8')
    reshaped_array[imgName] = new_img
  return reshaped_array

def blur_1(img, sd=1):
  new_img = skimage.filters.gaussian(img, sigma=sd)
  return new_img

def Scale_1(img):
  ratio = random.choice([0.1, 0.2, 0.3, 0.4])
  x = int(ratio * 600 / 2)
  scaled = img[x:600-x, x:600-x]
  new_img = cv2.resize(scaled, dsize=(600, 600), interpolation=cv2.INTER_CUBIC)
  return new_img
    

def Rotate30_1(img): 
  rand_dir = random.choice([-1, 1])
  new_img = rotate(img, rand_dir * 30) 
  return new_img

def VerticalFlip_1(img): 
  new_img = np.fliplr(img)
  return new_img

def Translation_1(img): 
  rand_x = random.randrange(-150, 150, 50)
  rand_y = random.randrange(-150, 150, 50)
  transform = AffineTransform(translation=(rand_x,rand_y))
  new_img = warp(img,transform, mode="constant")
  return new_img
####################################################


In [7]:
path = './handgesturedataset_part1/'
save_path = './Mult_Augments/'
# images in an array named imgs
imgs_png = loadImages(path)

# Step 1 convert png_images to np arrays
imgs_array_before = ImagesToArray(imgs_png)

# # Step 2 Reshape the images
imgs_array_reshaped = shape600(imgs_array_before)
images = imgs_array_reshaped
!mkdir ./Mult_Augments

## Adding Multiple Augmentations to each image in the Dataset

In [ ]:
# create test set with data augmentations
images = imgs_array_reshaped
for this_img in images.items(): 
  num_aug = random.choice([0, 1, 2, 3])
  new_img = this_img[1]
  img_save = Image.fromarray(new_img, 'RGB')
  img_save.save(save_path + this_img[0] + '_aug_temp' + '.png')
  
  num_list = list(range(0, 5)) 
  shuffled_list = random.shuffle(num_list)
  name = ''
  for n in range(num_aug): 
    this_aug = num_list[n]
    new_img = Image.open(save_path + this_img[0] + '_aug_temp' + '.png')
    new_img = np.array(new_img)

    if (this_aug == 0): 
      new_img = blur_1(new_img, 10)
      name += '_blurred'
      img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    elif (this_aug == 1): 
      new_img = Scale_1(new_img)
      name += '_scaled'
      img_save = Image.fromarray(new_img, 'RGB')
    elif (this_aug == 2): 
      new_img = Rotate30_1(new_img)
      name += '_rotated'
      img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    elif (this_aug == 3): 
      new_img = VerticalFlip_1(new_img)
      name += '_flipped'
      img_save = Image.fromarray(new_img, 'RGB')
    elif (this_aug == 4): 
      new_img = Translation_1(new_img)
      name += '_translated'
      img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    
    img_save.save(save_path + this_img[0] + '_aug_temp' + '.png')

  img_save.save(save_path + this_img[0] + '_aug' + name + '.png')

#remove all temp images
!rm ./Mult_Augments/*temp*
pass